In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
from dotenv import load_dotenv
import psycopg2
folder_path = 'excel files'


In [2]:
dataframes_dict = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)    
        df = pd.read_excel(file_path,None) 
        # print(df.values()) 
        dataframes_dict[filename] = df
        print(dataframes_dict[filename].keys()) 
        


dict_keys(['2022-23', '2023-24'])
dict_keys(['2022-23', '2023-24'])
dict_keys(['2022-23', 'Summary of 2022-23'])


In [3]:
dataframes_dict.values()

dict_values([{'2022-23':      Download File  id                                         Book Title  \
0       Click Here   1              LECTURE NOTES IN NETWORKS AND SYSTEMS   
1       Click Here   2  PROCEEDINGS OF THIRD INTERNATIONAL CONFERENCE ...   
2       Click Here   3  2022 IEEE BOMBAY SECTION SIGNATURE CONFERENCE ...   
3       Click Here   4              HORIZONS IN COMPUTER SCIENCE RESEARCH   
4       Click Here   5              HORIZONS IN COMPUTER SCIENCE RESEARCH   
5                    6  EXPLAINABLE EDGE AI: A FUTURISTIC COMPUTING PE...   
6       Click Here   7  CONSTRAINT DECISION-MAKING SYSTEMS IN ENGINEERING   
7       Click Here   8  PROCEEDINGS OF INTERNATIONAL CONFERENCE ON REC...   
8       Click Here   9  PROCEEDINGS OF INTERNATIONAL CONFERENCE ON REC...   
9       Click Here  10  PROCEEDINGS OF INTERNATIONAL CONFERENCE ON REC...   
10                  11              LECTURE NOTES IN NETWORKS AND SYSTEMS   
11                  12            LECTURE NOTES IN 

In [4]:
for key, value in dataframes_dict.items():
    print(key)
    for k,v in value.items():
        print(k)
        

Book_Chapter_Publication.xlsx
2022-23
2023-24
Conference_Publication.xlsx
2022-23
2023-24
Journal_Papers.xlsx
2022-23
Summary of 2022-23


In [5]:
del dataframes_dict['Journal_Papers.xlsx']['Summary of 2022-23']

In [6]:
for key, value in dataframes_dict.items():
    print(key)
    for k,v in value.items():
        print(k)
        

Book_Chapter_Publication.xlsx
2022-23
2023-24
Conference_Publication.xlsx
2022-23
2023-24
Journal_Papers.xlsx
2022-23


In [7]:
dataframes_dict['Book_Chapter_Publication.xlsx']['2022-23'].head()

,Download File,id,Book Title,Book Chapter Title,Name of Book Publisher,Month,Year,Year of Pub.,ISBNNo No,Publication Type,Publication City,Author,Department Name
0,Click Here,1,LECTURE NOTES IN NETWORKS AND SYSTEMS,CSBRCA: CLOUD SECURITY BREACHES AND ITS ROOT C...,SPRINGER,7,2022,2022-23,978-981-19-1142-2,Chapter in Book,Singapore,"CHUDASAMA VIPUL HASMUKHLAL, MADHURI BHAVSAR, V...",COMPUTER SCIENCE & ENG.DEPT.(UG)
1,Click Here,2,PROCEEDINGS OF THIRD INTERNATIONAL CONFERENCE ...,A SENTIMENT ANALYSIS-BASED RECOMMENDER FRAMEWO...,"SPRINGER, SINGAPORE",7,2022,2022-23,978-981-19-1141-5,Chapter in Proceeding,Singapore,"AKHIL BHATIA, SUDEEP TANWAR, ANANSHA ASTHANA",COMPUTER SCIENCE & ENG.DEPT.(UG)
2,Click Here,3,2022 IEEE BOMBAY SECTION SIGNATURE CONFERENCE ...,AI BASED CLASSIFICATION FOR AUTISM SPECTRUM DI...,IEEE,2,2023,2022-23,978-1-6654-9291-1,Chapter in Proceeding,Bombay,"VERMA JAIPRAKASH VINODKUMAR, SWATI JAIN, SHIVA...",COMPUTER SCIENCE & ENG.DEPT.(UG)
3,Click Here,4,HORIZONS IN COMPUTER SCIENCE RESEARCH,THE FUTURE OF INTRUSION DETECTION SYSTEM IN CL...,NOVA SCIENCE PUBLISHERS VOLUME 23,10,2022,2022-23,21592012,Chapter in Book,Newyork,SUNIL GAUTAM,COMPUTER SCIENCE & ENG.DEPT.(UG)
4,Click Here,5,HORIZONS IN COMPUTER SCIENCE RESEARCH,INTRUSION DETECTION SYSTEMS FOR ADVANCED ECOSY...,NOVA SCIENCE PUBLISHERS VOLUME 23,10,2022,2022-23,21592012,Chapter in Book,Newyork,SUNIL GAUTAM,COMPUTER SCIENCE & ENG.DEPT.(UG)


In [8]:
excel_files=list(dataframes_dict.keys())


In [9]:
def split_authors(df, column_name, max_authors=10):
    if column_name in df.columns:
        df.loc[:,column_name] = df[column_name].str.strip().str.replace(r',+', ',', regex=True).str.rstrip(',')
    else:
        return df

    # Split the 'author' column into lists of authors
    authors_split = df[column_name].str.split(',', expand=True)
    print(authors_split.shape[0])
    print(authors_split.shape[1])
    # Rename columns to 'author_1', 'author_2', ..., 'author_10'
    authors_split.columns = [f'{column_name}{i+1}' for i in range(authors_split.shape[1])]
    
    # If there are less columns than max_authors, add additional columns with None
    for i in range(authors_split.shape[1], max_authors):
        authors_split[f'{column_name}{i+1}'] = None
    
    # Trim to max_authors columns
    authors_split = authors_split.iloc[:, :max_authors]
    
    # Drop the original 'author' column and concatenate the new columns
    df = df.drop(columns=[column_name])
    df = pd.concat([df, authors_split], axis=1)
    
    return df
df = split_authors(dataframes_dict['Book_Chapter_Publication.xlsx']['2022-23'], 'Author', max_authors=10)


42
6


In [10]:
df.head()

,Download File,id,Book Title,Book Chapter Title,Name of Book Publisher,Month,Year,Year of Pub.,ISBNNo No,Publication Type,...,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Author8,Author9,Author10
0,Click Here,1,LECTURE NOTES IN NETWORKS AND SYSTEMS,CSBRCA: CLOUD SECURITY BREACHES AND ITS ROOT C...,SPRINGER,7,2022,2022-23,978-981-19-1142-2,Chapter in Book,...,CHUDASAMA VIPUL HASMUKHLAL,MADHURI BHAVSAR,VIVEK KUMAR PRASAD,ASHEESH SHAH,AKSHAY MEWADA,None,None,None,None,None
1,Click Here,2,PROCEEDINGS OF THIRD INTERNATIONAL CONFERENCE ...,A SENTIMENT ANALYSIS-BASED RECOMMENDER FRAMEWO...,"SPRINGER, SINGAPORE",7,2022,2022-23,978-981-19-1141-5,Chapter in Proceeding,...,AKHIL BHATIA,SUDEEP TANWAR,ANANSHA ASTHANA,None,None,None,None,None,None,None
2,Click Here,3,2022 IEEE BOMBAY SECTION SIGNATURE CONFERENCE ...,AI BASED CLASSIFICATION FOR AUTISM SPECTRUM DI...,IEEE,2,2023,2022-23,978-1-6654-9291-1,Chapter in Proceeding,...,VERMA JAIPRAKASH VINODKUMAR,SWATI JAIN,SHIVANI PANDYA,None,None,None,None,None,None,None
3,Click Here,4,HORIZONS IN COMPUTER SCIENCE RESEARCH,THE FUTURE OF INTRUSION DETECTION SYSTEM IN CL...,NOVA SCIENCE PUBLISHERS VOLUME 23,10,2022,2022-23,21592012,Chapter in Book,...,SUNIL GAUTAM,None,None,None,None,None,None,None,None,None
4,Click Here,5,HORIZONS IN COMPUTER SCIENCE RESEARCH,INTRUSION DETECTION SYSTEMS FOR ADVANCED ECOSY...,NOVA SCIENCE PUBLISHERS VOLUME 23,10,2022,2022-23,21592012,Chapter in Book,...,SUNIL GAUTAM,None,None,None,None,None,None,None,None,None


In [11]:
load_dotenv()


True

In [12]:
def drop_database(db_name):
    # Connect to the 'postgres' database as a superuser or a user with sufficient privileges
    conn = psycopg2.connect(
        dbname='postgres',
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host=os.getenv('DB_HOST'),
        port=os.getenv('DB_PORT')
    )
    conn.autocommit = True
    cursor = conn.cursor()
    
    try:
        cursor.execute(f"DROP DATABASE IF EXISTS \"{db_name}\"")
        print(f"Database \"{db_name}\" dropped successfully.")
    except Exception as e:
        print(f"Error dropping database \"{db_name}\": {e}")
    finally:
        cursor.close()
        conn.close()





# Function to check if the database exists
def database_exists(db_name):
    conn = psycopg2.connect(
        dbname='postgres',
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host=os.getenv('DB_HOST'),
        port=os.getenv('DB_PORT')
    )
    conn.autocommit = True
    cursor = conn.cursor()
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{db_name}'")
    exists = cursor.fetchone() is not None
    cursor.close()
    conn.close()
    return exists



# Function to create the database if it doesn't exist
def create_database(db_name):
    if not database_exists(db_name):
        conn = psycopg2.connect(
            dbname='postgres',
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            host=os.getenv('DB_HOST'),
            port=os.getenv('DB_PORT')
        )
        conn.autocommit = True
        cursor = conn.cursor()
        cursor.execute(f"CREATE DATABASE {db_name}")
        print(f"Database {db_name} created successfully.")
        cursor.close()
        conn.close()
    else:
        print(f"Database {db_name} already exists.")
        drop_database(db_name)
        create_database(db_name)


# Function to create a PostgreSQL database and insert data into tables
def create_db_and_insert_data(db_name, sheet_data):
    # Create a connection to PostgreSQL
    default_engine = create_engine(f'postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/postgres')
    
    # Try to create the database if it doesn't exist
    try:
        create_database(db_name)
    except OperationalError as e:
        if "already exists" in str(e):
            print(f"Database {db_name} already exists.")
            
        else:
            print(e)
            
    
    # Create an engine for the new database
    engine = create_engine(f'postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{db_name}')
    
    # Loop through each sheet and create a table in the database
    for sheet_name, df in sheet_data.items():
        print(sheet_name)
        df.to_sql(sheet_name, engine, if_exists='replace', index=False)

In [13]:
for i in excel_files:
    for df in dataframes_dict[i]:
        print(i)
        dataframes_dict[i][df]=split_authors(dataframes_dict[i][df], 'Author', max_authors=10)

Book_Chapter_Publication.xlsx
42
6
Book_Chapter_Publication.xlsx
28
5
Conference_Publication.xlsx
Conference_Publication.xlsx
49
7
Journal_Papers.xlsx


In [14]:
for i in excel_files:
    create_db_and_insert_data(i[0:-5].lower(),dataframes_dict[i])

Database book_chapter_publication already exists.
Database "book_chapter_publication" dropped successfully.
Database book_chapter_publication created successfully.
2022-23
2023-24
Database conference_publication already exists.
Database "conference_publication" dropped successfully.
Database conference_publication created successfully.
2022-23
2023-24
Database journal_papers already exists.
Database "journal_papers" dropped successfully.
Database journal_papers created successfully.
2022-23


In [15]:
# SELECT 'CREATE DATABASE exp_db' 
# WHERE NOT EXISTS (SELECT FROM pg_database WHERE datname = 'exp_db')